In [1]:
import arcpy, requests, datetime
import xml.etree.ElementTree as ET
from arcgis.gis import GIS
g = GIS("https://www.arcgis.com", "linds-m", "linds-m&2210")

try:
    arcpy.env.overwriteOutput = True
    fs = g.content.search(query='title:Community_Trees', item_type = "Feature Layer", max_items=1)
    url = fs[0].layers[0].url
    fields = ['Planted_date',
              'TreeLocation_Long',
              'TreeLocation_Lat',
              'SpeciesCode',
              'DBH_CM',
              'HouseVintage',
              'DirectionDegree',
              'DistanceMeter',
              'Heated',
              'AirConditioned',
              'EcoBen_RunoffAvoided',
              'EcoBen_RunoffAvoidedValue',
              'EcoBen_HydroIntercep',
              'EcoBen_CORemoved',
              'EcoBen_CORemovedValue',
              'EcoBen_NO2Removed',
              'EcoBen_NO2RemovedValue',
              'EcoBen_SO2Removed',
              'EcoBen_SO2RemovedValue',
              'EcoBen_PM25Removed',
              'EcoBen_PM25RemovedValue',
              'EcoBen_O3Removed',
              'EcoBen_O3RemovedValue',
              'EcoBen_COAvoided',
              'EcoBen_COAvoidedValue',
              'EcoBen_NO2Avoided',
              'EcoBen_NO2AvoidedValue',
              'EcoBen_SO2Avoided',
              'EcoBen_SO2AvoidedValue',
              'EcoBen_PM25Avoided',
              'EcoBen_PM25AvoidedValue',
              'EcoBen_VOCAvoided',
              'EcoBen_VOCAvoidedValue',
              'EcoBen_CO2Sequest',
              'EcoBen_CO2SequestValue',
              'EcoBen_CO2Avoided',
              'EcoBen_CO2AvoidedValue',
              'EcoBen_CarbonStorage',
              'EcoBen_CO2Storage',
              'EcoBen_CO2StorageValue',
              'EcoBen_HeatingFuelValue',
              'EcoBen_ElectricityHeatValue',
              'EcoBen_ElectricityCoolValue'
                ]
    print('Running trees through the iTree API to calculate Eco Benefits...')
    with arcpy.da.UpdateCursor(url, fields) as cursor:
        
        for row in cursor:

            api_url = 'https://dtbe-api.daveyinstitute.com/v2/getGrowoutTreeBenefit'
            PARAMS = {'key': '272752b916654994a7e781cd037fc4ed',
                        'ProjectYears':'1',
                        'MortalityRate':'0.00',
                        'Longitude':row[1],
                        'Latitude':row[2],
                        'TreeAmount':'1',
                        'Species':row[3],
                        'DBHCentimeter':row[4],
                        'TreeHeightMeter':'-1',
                        'TreeCrownHeightMeter':'-1',
                        'TreeCrownWidthMeter':'-1',
                        'Condition':'Good',
                        'CLE':'3',
                        'EnergyOnly':'0',
                        'HouseVintage':row[5],
                        'DirectionDegree':row[6],
                        'DistanceMeter':row[7],
                        'Heated':row[8],
                        'AirConditioned':row[9]
                    }

            response = requests.get(url = api_url, params = PARAMS)
            root = ET.fromstring(response.content)

            if response.status_code == 200:
                print('Successful i-Tree Engine API Call')

                for child in root.iter('*'):
                    if child.tag == 'RunoffAvoided':
                        row[10] = child.text
                    elif child.tag == 'RunoffAvoidedValue':
                        row[11] = child.text
                    elif child.tag == 'Interception':
                        row[12] = child.text
                    elif child.tag == 'O3':
                        row[21] = child.text
                    elif child.tag == 'O3Value':
                        row[22] = child.text
                    elif child.tag == 'VOC':
                        row[31] = child.text
                    elif child.tag == 'VOCValue':
                        row[32] = child.text
                    elif child.tag == 'MbtuHeatClimateValue':
                        row[40] = child.text
                    elif child.tag == 'KWhElectricityHeatClimateValue':
                        row[41] = child.text
                    elif child.tag == 'KWhElectricityCoolClimateValue':
                        row[42] = child.text

                for child in root.find('.//AirQualityBenefit_PollRemoved'):
                    if child.tag == 'CO':
                        row[13] = child.text
                    elif child.tag == 'COValue':
                        row[14] = child.text
                    elif child.tag == 'NO2':
                        row[15] = child.text
                    elif child.tag == 'NO2Value':
                        row[16] = child.text
                    elif child.tag == 'SO2':
                        row[17] = child.text
                    elif child.tag == 'SO2Value':
                        row[18] = child.text
                    elif child.tag == 'PM25':
                        row[19] = child.text
                    elif child.tag == 'PM25Value':
                        row[20] = child.text

                for child in root.find('.//AirQualityBenefit_PollAvoided'):
                    if child.tag == 'CO':
                        row[23] = child.text
                    elif child.tag == 'COValue':
                        row[24] = child.text
                    elif child.tag == 'NO2':
                        row[25] = child.text
                    elif child.tag == 'NO2Value':
                        row[26] = child.text
                    elif child.tag == 'SO2':
                        row[27] = child.text
                    elif child.tag == 'SO2Value':
                        row[28] = child.text
                    elif child.tag == 'PM25':
                        row[29] = child.text
                    elif child.tag == 'PM25Value':
                        row[30] = child.text

                for child in root.find('.//CO2Benefit_Sequestered'):
                    if child.tag == 'CO2':
                        row[33] = child.text
                    elif child.tag == 'CO2Value':
                        row[34] = child.text

                for child in root.find('.//CO2Benefit_Avoided'):
                    if child.tag == 'CO2':
                        row[35] = child.text
                    elif child.tag == 'CO2Value':
                        row[36] = child.text

                for child in root.find('.//TreeCarbon'):
                    if child.tag == 'CarbonStorage':
                        row[37] = child.text
                    elif child.tag == 'CarbonDioxideStorage':
                        row[38] = child.text
                    elif child.tag == 'CarbonDioxideStorageValue':
                        row[39] = child.text

                        cursor.updateRow(row)

    print('Done adding eco benefit values to the attribute table')

    print('Calculating fields...')
    arcpy.management.CalculateField(url, 'Calc_CarbonDRemovalValue', '!EcoBen_CO2SequestValue! + !EcoBen_CO2AvoidedValue!', 'PYTHON3')
    arcpy.management.CalculateField(url, 'Calc_RunoffAvoidedGallons', '!EcoBen_RunoffAvoided! * 264.172', 'PYTHON3')
    arcpy.management.CalculateField(url, 'Calc_PollutantsRemoved', '(!EcoBen_CORemoved! + !EcoBen_NO2Removed! + !EcoBen_SO2Removed! + !EcoBen_PM25Removed! + !EcoBen_O3Removed!) * 2.20462', 'PYTHON3')
    arcpy.management.CalculateField(url, 'Calc_PollutantsRemovedValue', '!EcoBen_CORemovedValue! + !EcoBen_NO2RemovedValue! + !EcoBen_SO2RemovedValue! + !EcoBen_PM25RemovedValue! + !EcoBen_O3RemovedValue!', 'PYTHON3')
    arcpy.management.CalculateField(url, 'Calc_EnergySavings', '!EcoBen_HeatingFuelValue! + !EcoBen_ElectricityHeatValue! + !EcoBen_ElectricityCoolValue!', 'PYTHON3')
    print('Done calculating fields.')

    print('DONE!')

except Exception as e:
    print('Uh oh. Something went wrong!')
    print('Error: ' + e.args[0])

Running trees through the iTree API to calculate Eco Benefits...
Successful i-Tree Engine API Call
Successful i-Tree Engine API Call
Successful i-Tree Engine API Call
Successful i-Tree Engine API Call
Done adding eco benefit values to the attribute table
Calculating fields...
Uh oh. Something went wrong!
Error: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Error code: 503
Description: Service Unavailable.
Message: An error occurred.
Failed to execute (CalculateField).

